# Introduction

In this project we will try to find the best food locations in New York City, so that food tourists can experience the city at its best.

Since there are lots of restaurants in New York and it is a big city we will try to detect **boroughs that have many high rated venues** as well as a **variety of food** so that people do not have to travel all over the city.

# Data

Based on definition of our problem, factors that will influence our decision are:
* rating of food available in a given borough
* variety of food available in a given borough

Following data sources will be needed to generate the required information:
* restaurants, their type and ratings in every borough will be obtained using **Foursquare API**

# Installing Prereqs

In [1]:
# install geocoder:
!conda install -c conda-forge geocoder --yes
print('geocoder installed!')

Solving environment: done

# All requested packages already installed.

geocoder installed!


In [2]:
# install folium:
!conda install -c conda-forge folium=0.5.0
print('folium installed!')

Solving environment: done

# All requested packages already installed.

folium installed!


# Importing packages

In [3]:
import numpy as np # library to handle data

import pandas as pd # library for data analysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library for JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library for requests
from pandas.io.json import json_normalize # make JSON file into a pandas dataframe

# Matplotlib
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means
from sklearn.cluster import KMeans

import folium # map library

print('Everything imported.')

Everything imported.


# Getting Dataset

# Foursquare API Credentials

In [4]:
CLIENT_ID = 'AVKHHHIOUC5110XTQMRCUG12EBOWZ5XY5GANGRM0IKZJMRHX' # your Foursquare ID
CLIENT_SECRET = 'WW3QQWNMADJX551D5YKVQSDIOIWUI05G2YHWAGJRRUI0JTSC' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentials:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentials:
CLIENT_ID: AVKHHHIOUC5110XTQMRCUG12EBOWZ5XY5GANGRM0IKZJMRHX
CLIENT_SECRET:WW3QQWNMADJX551D5YKVQSDIOIWUI05G2YHWAGJRRUI0JTSC


The five boroughs of New York are well known. First, let's get the coordinates for them to feed into the FourSquare API.

In [5]:
boroughs = ["Bronx","Brooklyn","Staten Island","Manhattan","Queens"]

In [12]:
# define the dataframe columns
column_names = ['Borough', 'Latitude', 'Longitude'] 

# make the dataframe
borough_df = pd.DataFrame(columns=column_names)

In [13]:
#populate borough_df
for name in boroughs:
    address = name + ', NY'
    geolocator = Nominatim(user_agent="ny_explorer")
    location = geolocator.geocode(address)
    lat = location.latitude
    lon = location.longitude
    borough_df = borough_df.append({'Borough': name,
                                          'Latitude': lat,
                                          'Longitude': lon}, ignore_index=True)
    
    print('The geograpical coordinates of ' + address + ' is {}, {}.'.format(lat, lon))

The geograpical coordinates of Bronx, NY is 40.8466508, -73.8785937.
The geograpical coordinates of Brooklyn, NY is 40.6501038, -73.9495823.
The geograpical coordinates of Staten Island, NY is 40.5834557, -74.1496048.
The geograpical coordinates of Manhattan, NY is 40.7896239, -73.9598939.
The geograpical coordinates of Queens, NY is 40.7498243, -73.7976337.


In [14]:
borough_df

,Borough,Latitude,Longitude
0,Bronx,40.846651,-73.878594
1,Brooklyn,40.650104,-73.949582
2,Staten Island,40.583456,-74.149605
3,Manhattan,40.789624,-73.959894
4,Queens,40.749824,-73.797634


Now we need to find the places we can get food in each of these boroughs.

In [55]:
# define the dataframe columns
venue_column_names = ['city','id','name','categories','lat','lng']

# make the dataframe
venue_df = pd.DataFrame(columns=venue_column_names)

In [56]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [57]:
for i in range(len(borough_df)):
    borough_latitude = borough_df.loc[i, 'Latitude'] # borough latitude value
    borough_longitude = borough_df.loc[i, 'Longitude'] # borough longitude value
    borough_name = borough_df.loc[i, 'Borough'] # borough name
    section = "food" #limits types of venues returned by Foursquare API
    LIMIT = 25 # limit of number of venues returned by Foursquare API, only pulling 25 because I'm accessing a premium endpoint
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&section={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    borough_latitude, 
    borough_longitude,
    section,
    LIMIT)
    results = requests.get(url).json()
    venues = results['response']['groups'][0]['items']
    venues
    nearby_venues = json_normalize(venues) # flatten JSON
    filtered_columns = ['venue.location.city','venue.id', 'venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
    nearby_venues =nearby_venues.loc[:, filtered_columns]
    nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)
    nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
    venue_df = venue_df.append(nearby_venues, ignore_index=True)

In [58]:
venue_df

,city,id,name,categories,lat,lng
0,Bronx,4d39a9bacc48224b57ae354f,North Star Cafe,Café,40.843336,-73.886982
1,Bronx,4bc33005abf49521fe5cc393,Joes Italian Deli,Deli / Bodega,40.854391,-73.885367
2,Bronx,4af0fa29f964a52060e021e3,Antonio's Trattoria,Italian Restaurant,40.854083,-73.886616
3,Bronx,4ace366bf964a5201bcf20e3,Mike's Deli,Deli / Bodega,40.854131,-73.888136
4,Bronx,4ae8732df964a5206daf21e3,Casa Della Mozzarella,Deli / Bodega,40.855440,-73.887373
5,Bronx,4a8e261cf964a5201c1220e3,Roberto's,Italian Restaurant,40.853607,-73.888430
6,Bronx,4aabd3e6f964a5204a5a20e3,Zero Otto Nove,Pizza Place,40.854714,-73.888388
7,Bronx,4acf80aef964a52025d420e3,Tino's Delicatessen,Italian Restaurant,40.855882,-73.887166
8,Bronx,4bb6470cef159c7465b275f7,Calabria Pork Store,Italian Restaurant,40.854145,-73.888776
9,Bronx,4bb6496e46d4a59398fdc5c0,Madonia Bakery,Bakery,40.854412,-73.888516


Now we will get the ratings for each of these restaurants.

In [59]:
# define the dataframe columns
venue_rating_column_names = ['city','id','name','category','rating'] 

# make the dataframe
venue_rating_df = pd.DataFrame(columns=venue_rating_column_names)

In [60]:
for a in range(len(venue_df)):
    VENUE_ID = venue_df.loc[a, 'id']    
    rating_url = 'https://api.foursquare.com/v2/venues/'+VENUE_ID+'?client_id={}&client_secret={}&v={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION)
    rating_results = requests.get(rating_url).json()
    venue_details = rating_results['response']['venue']
    venue_city = venue_details['location']['city']
    venue_id = venue_details['id']
    venue_name = venue_details['name']
    venue_category = venue_details['categories'][0]['name']
    venue_rating = venue_details['rating']
    venue_rating_df = venue_rating_df.append({'city': venue_city,
                                              'id': venue_id,
                                              'name': venue_name,
                                              'category': venue_category,
                                              'rating': venue_rating}, ignore_index=True)

In [61]:
venue_rating_df

,city,id,name,category,rating
0,Bronx,4d39a9bacc48224b57ae354f,North Star Cafe,Café,8.3
1,Bronx,4bc33005abf49521fe5cc393,Joes Italian Deli,Deli / Bodega,8.9
2,Bronx,4af0fa29f964a52060e021e3,Antonio's Trattoria,Italian Restaurant,9.0
3,Bronx,4ace366bf964a5201bcf20e3,Mike's Deli,Deli / Bodega,8.9
4,Bronx,4ae8732df964a5206daf21e3,Casa Della Mozzarella,Deli / Bodega,9.1
5,Bronx,4a8e261cf964a5201c1220e3,Roberto's,Italian Restaurant,8.6
6,Bronx,4aabd3e6f964a5204a5a20e3,Zero Otto Nove,Pizza Place,9.0
7,Bronx,4acf80aef964a52025d420e3,Tino's Delicatessen,Italian Restaurant,9.2
8,Bronx,4bb6470cef159c7465b275f7,Calabria Pork Store,Italian Restaurant,8.8
9,Bronx,4bb6496e46d4a59398fdc5c0,Madonia Bakery,Bakery,8.8


Let's split them into boroughs to compare them, and also drop any fast food restaurants that I recognized.

In [117]:
bronx_df = venue_rating_df[0:25]
bronx_df = bronx_df.sort_values(by= ['category','rating'], ascending=False)
bronx_df

,city,id,name,category,rating
6,Bronx,4aabd3e6f964a5204a5a20e3,Zero Otto Nove,Pizza Place,9.0
20,Bronx,4bc78c0d0050b71372e8b83b,John & Joe's Pizzeria,Pizza Place,8.7
11,Bronx,4aee1220f964a520ced121e3,Full Moon Pizzeria,Pizza Place,8.6
10,Bronx,4b82c608f964a52001e430e3,Morris Park Pizza,Pizza Place,8.0
23,Bronx,4fcfeb586b748e9d19eac30d,Arth Aljanathain,Middle Eastern Restaurant,7.8
15,Bronx,4c27e5e0fe6e2d7f4f84523c,Estrellita Poblana,Mexican Restaurant,8.4
19,New York,4cf3f09c7bf3b60c5daa6d7f,Angel's Grill,Latin American Restaurant,7.8
7,Bronx,4acf80aef964a52025d420e3,Tino's Delicatessen,Italian Restaurant,9.2
2,Bronx,4af0fa29f964a52060e021e3,Antonio's Trattoria,Italian Restaurant,9.0
8,Bronx,4bb6470cef159c7465b275f7,Calabria Pork Store,Italian Restaurant,8.8


In [128]:
brooklyn_df = venue_rating_df[25:50]
brooklyn_df = brooklyn_df.sort_values(by= ['category','rating'], ascending=False)
brooklyn_df = brooklyn_df.drop(brooklyn_df[brooklyn_df.name.isin(["IHOP"])].index)
brooklyn_df

,city,id,name,category,rating
39,Brooklyn,4a89d100f964a520160920e3,Zen Vegetarian House,Vegetarian / Vegan Restaurant,8.2
42,Brooklyn,4c261b725c5ca5939d7845fe,Mama Louisa's Hero Shoppe,Sandwich Place,8.3
36,New York,59cfb148e2d4aa3bd74f6d80,Risbo,Restaurant,9.1
35,Brooklyn,564fc417498e58f2bfed7138,ZuriLee Pizza Bar,Pizza Place,8.5
44,Brooklyn,4a79b761f964a52070e71fe3,Gino's Pizzeria,Pizza Place,7.8
40,Brooklyn,59bfcdfb2079551e2b628d20,Camillo,Italian Restaurant,8.5
31,Brooklyn,4bb93b70cf2fc9b6fe64a002,Gandhi Fine Indian Cuisine,Indian Restaurant,8.5
45,Brooklyn,573f7a57498eec371cec96c2,Empanada City,Empanada Restaurant,9.2
28,Brooklyn,4ae722c6f964a520bea821e3,Nostrand Donut Shop,Donut Shop,7.6
43,Brooklyn,4bfafc771134b713e48c0fc9,Green Lake Chinese Restaurant,Chinese Restaurant,7.9


In [130]:
staten_island_df = venue_rating_df[50:75]
staten_island_df = staten_island_df.sort_values(by= ['category','rating'], ascending=False)
staten_island_df = staten_island_df.drop(staten_island_df[staten_island_df.name.isin(["Outback Steakhouse", "Jersey Mike\'s Subs", "Chipotle Mexican Grill", "Sarku Japan", "Dunkin\'", "Shake Shack", "Panera Bread"])].index)
staten_island_df

,city,id,name,category,rating
71,Staten Island,4b75af66f964a520e31c2ee3,Sanma Japanese Restaurant,Sushi Restaurant,8.2
66,Staten Island,4b22e2a3f964a5207c4f24e3,Canlon's,Restaurant,8.1
57,Staten Island,5480e64f498ebc7397d9e763,Miller's Ale House,Restaurant,7.8
59,Staten Island,4b85d75ff964a520cb7531e3,Holy Schnitzel,Restaurant,7.5
67,Staten Island,4db59a820437fa536a4429b1,Pizzeria Giove,Pizza Place,8.5
60,Staten Island,4b5b9fc8f964a520ca0b29e3,Gino's Villa Monte Pizzeria & Restaurant,Pizza Place,7.6
74,Staten Island,4b343839f964a5203a2625e3,La Strada,Italian Restaurant,7.9
54,Staten Island,5079ccdce4b065f1dd8155ad,Casa verde,Italian Restaurant,7.8
63,Staten Island,588a0ec3f5bfab287dba9a15,angelinas kitchen,Italian Restaurant,7.4
72,Staten Island,4c27cc8f3492a593cb72b628,Classic Pizza,Italian Restaurant,7.4


In [132]:
manhattan_df = venue_rating_df[75:100]
manhattan_df = manhattan_df.sort_values(by= ['category','rating'], ascending=False)
manhattan_df = manhattan_df.drop(manhattan_df[manhattan_df.name.isin(["Chipotle Mexican Grill"])].index)
manhattan_df

,city,id,name,category,rating
97,New York,4b4f59def964a520a20227e3,Champignon,Sandwich Place,7.1
79,New York,4b38f360f964a520e55225e3,Paola's Restaurant,Restaurant,8.4
92,New York,4b06e4bff964a52061f222e3,Little Luzzo's Pizza,Pizza Place,7.7
88,New York,4bf41dcae5eba593cd441f90,Marco Polo Pizza & Deli,Pizza Place,7.6
75,New York,56cc9aa6498edc84f39f1cfd,Russ & Daughters,Kosher Restaurant,8.8
89,New York,4bb4d86886a8d13a83f8de7f,Sfoglia,Italian Restaurant,8.2
99,New York,5914f7b29de23b672db8bf58,Trattoria Il Gusto,Italian Restaurant,7.6
86,New York,4b7f14d5f964a520191530e3,Tre Otto,Italian Restaurant,7.5
91,New York,58532c7006f1a339fd294451,Effy's Cafe,Israeli Restaurant,7.8
81,New York,4b9d2218f964a5207e9236e3,Table d'Hote,French Restaurant,8.3


In [134]:
queens_df = venue_rating_df[100:125]
queens_df = queens_df.sort_values(by= ['category','rating'], ascending=False)
queens_df = queens_df.drop(queens_df[queens_df.name.isin(["QDOBA Mexican Eats", "Arby\'s"])].index)
queens_df

,city,id,name,category,rating
109,Queens,5a822e1f9e0d540546c6ee2c,Pho Che,Vietnamese Restaurant,7.9
106,Bayside,49fdbd6cf964a520256f1fe3,BKNY Thai Restaurant,Thai Restaurant,8.5
113,Flushing,56231f12498e732a612e9896,童年时代 Time Flies Bbq & Bar,Seafood Restaurant,7.9
100,Flushing,4bd3afcd462cb713701fdf07,Pizza Garden,Pizza Place,8.8
117,Fresh Meadows,4d7576740f588eecc064c7c2,Brother's Pizza,Pizza Place,7.8
116,Bayside,4b53e4b4f964a52024ae27e3,Fontana Famous Pizza & Gyro,Modern Greek Restaurant,8.1
107,Flushing,49e10e0bf964a52089611fe3,Mad For Chicken,Korean Restaurant,9.1
121,Flushing,4b830e44f964a520ebf430e3,Hahm Ji Bach - 함지박,Korean Restaurant,8.7
103,Flushing,4eefef69be7ba3ed7b202455,Arirang,Korean Restaurant,8.5
124,Flushing,4b8f0931f964a520d04533e3,Mapo BBQ,Korean Restaurant,8.5


Now let's take only the best of each category

In [148]:
bronx_best_of = bronx_df.sort_values('rating',ascending = False).groupby('category').head(1)
bronx_best_of

,city,id,name,category,rating
7,Bronx,4acf80aef964a52025d420e3,Tino's Delicatessen,Italian Restaurant,9.2
4,Bronx,4ae8732df964a5206daf21e3,Casa Della Mozzarella,Deli / Bodega,9.1
6,Bronx,4aabd3e6f964a5204a5a20e3,Zero Otto Nove,Pizza Place,9.0
9,Bronx,4bb6496e46d4a59398fdc5c0,Madonia Bakery,Bakery,8.8
15,Bronx,4c27e5e0fe6e2d7f4f84523c,Estrellita Poblana,Mexican Restaurant,8.4
0,Bronx,4d39a9bacc48224b57ae354f,North Star Cafe,Café,8.3
19,New York,4cf3f09c7bf3b60c5daa6d7f,Angel's Grill,Latin American Restaurant,7.8
23,Bronx,4fcfeb586b748e9d19eac30d,Arth Aljanathain,Middle Eastern Restaurant,7.8


In [150]:
brooklyn_best_of = brooklyn_df.sort_values('rating',ascending = False).groupby('category').head(1)
brooklyn_best_of

,city,id,name,category,rating
45,Brooklyn,573f7a57498eec371cec96c2,Empanada City,Empanada Restaurant,9.2
36,New York,59cfb148e2d4aa3bd74f6d80,Risbo,Restaurant,9.1
38,Brooklyn,5389f295498e264f18c5d652,PLG Coffee House and Tavern,Café,9.1
34,New York,4a6cba95f964a52073d11fe3,Peppa's Jerk Chicken,Caribbean Restaurant,8.9
35,Brooklyn,564fc417498e58f2bfed7138,ZuriLee Pizza Bar,Pizza Place,8.5
40,Brooklyn,59bfcdfb2079551e2b628d20,Camillo,Italian Restaurant,8.5
31,Brooklyn,4bb93b70cf2fc9b6fe64a002,Gandhi Fine Indian Cuisine,Indian Restaurant,8.5
48,Brooklyn,4b47bad2f964a5203d3c26e3,Allan's Bakery,Bakery,8.4
42,Brooklyn,4c261b725c5ca5939d7845fe,Mama Louisa's Hero Shoppe,Sandwich Place,8.3
39,Brooklyn,4a89d100f964a520160920e3,Zen Vegetarian House,Vegetarian / Vegan Restaurant,8.2


In [152]:
staten_island_best_of = staten_island_df.sort_values('rating',ascending = False).groupby('category').head(1)
staten_island_best_of

,city,id,name,category,rating
50,Staten Island,5b9db00f73fe25002c5982c2,Tommy's Tavern + Tap,Gastropub,8.7
61,Staten Island,5777f6eb498ecac8cfb24a82,Tropical Smoothie Cafe,Café,8.7
67,Staten Island,4db59a820437fa536a4429b1,Pizzeria Giove,Pizza Place,8.5
53,Staten Island,4bd041ad41b9ef3b98b1f9e5,Holtermanns,Bakery,8.4
71,Staten Island,4b75af66f964a520e31c2ee3,Sanma Japanese Restaurant,Sushi Restaurant,8.2
66,Staten Island,4b22e2a3f964a5207c4f24e3,Canlon's,Restaurant,8.1
70,Staten Island,4bac9a50f964a52093fd3ae3,Plaza Bagels & Deli,Bagel Shop,8.0
74,Staten Island,4b343839f964a5203a2625e3,La Strada,Italian Restaurant,7.9
73,Staten Island,4c320ad366e40f4767d3c58b,A & S Salumeria and Bakery,Deli / Bodega,7.8
64,Staten Island,50be4c36e4b0d20f31497336,Crepes Du Nord,French Restaurant,7.3


In [155]:
manhattan_best_of = manhattan_df.sort_values('rating',ascending = False).groupby('category').head(1)
manhattan_best_of

,city,id,name,category,rating
78,New York,4d1e8f8e5acaa35dfb7eb835,Earl's Beer & Cheese,American Restaurant,9.1
76,New York,5328a418498e4be839240b62,Da Capo,Café,9.0
75,New York,56cc9aa6498edc84f39f1cfd,Russ & Daughters,Kosher Restaurant,8.8
83,New York,49fa2837f964a520cf6d1fe3,Café Sabarsky,Austrian Restaurant,8.6
79,New York,4b38f360f964a520e55225e3,Paola's Restaurant,Restaurant,8.4
81,New York,4b9d2218f964a5207e9236e3,Table d'Hote,French Restaurant,8.3
80,New York,3fd66200f964a520d1e81ee3,Sarabeth's East,Breakfast Spot,8.2
89,New York,4bb4d86886a8d13a83f8de7f,Sfoglia,Italian Restaurant,8.2
93,New York,4a67b65ff964a520f2c91fe3,Joy Burger Bar,Burger Joint,8.1
91,New York,58532c7006f1a339fd294451,Effy's Cafe,Israeli Restaurant,7.8


In [157]:
queens_best_of = queens_df.sort_values('rating',ascending = False).groupby('category').head(1)
queens_best_of

,city,id,name,category,rating
107,Flushing,49e10e0bf964a52089611fe3,Mad For Chicken,Korean Restaurant,9.1
100,Flushing,4bd3afcd462cb713701fdf07,Pizza Garden,Pizza Place,8.8
102,Fresh Meadows,4ab5017ef964a520f37120e3,Bagel Oasis,Bagel Shop,8.7
105,Flushing,4d8e2d1c5091a1cddb7d8901,Ganesh Temple Canteen,Indian Restaurant,8.7
106,Bayside,49fdbd6cf964a520256f1fe3,BKNY Thai Restaurant,Thai Restaurant,8.5
111,Flushing,4b23d8fbf964a520215b24e3,Mythos,Greek Restaurant,8.1
116,Bayside,4b53e4b4f964a52024ae27e3,Fontana Famous Pizza & Gyro,Modern Greek Restaurant,8.1
118,Flushing,4b96ab2af964a520bfdc34e3,Durso's Pasta & Ravioli Company,Italian Restaurant,8.0
113,Flushing,56231f12498e732a612e9896,童年时代 Time Flies Bbq & Bar,Seafood Restaurant,7.9
109,Queens,5a822e1f9e0d540546c6ee2c,Pho Che,Vietnamese Restaurant,7.9


In [161]:
manhattan_rating = manhattan_best_of["rating"].mean()
manhattan_variety = len(manhattan_best_of.index)
manhattan_max = manhattan_best_of["rating"].max()
print('There are {} kinds of restaurants in Manhattan with an average rating of {} and a high of {}'.format(manhattan_variety, manhattan_rating, manhattan_max))

There are 14 kinds of restaurants in Manhattan with an average rating of 8.2 and a high of 9.1


In [158]:
brooklyn_rating = brooklyn_best_of["rating"].mean()
brooklyn_variety = len(brooklyn_best_of.index)
brooklyn_max = brooklyn_best_of["rating"].max()
print('There are {} kinds of restaurants in Brooklyn with an average rating of {} and a high of {}'.format(brooklyn_variety, brooklyn_rating, brooklyn_max))

There are 12 kinds of restaurants in Brooklyn with an average rating of 8.516666666666667 and a high of 9.2


In [162]:
queens_rating = queens_best_of["rating"].mean()
queens_variety = len(queens_best_of.index)
queens_max = queens_best_of["rating"].max()
print('There are {} kinds of restaurants in Queens with an average rating of {} and a high of {}'.format(queens_variety, queens_rating, queens_max))

There are 12 kinds of restaurants in Queens with an average rating of 8.225000000000001 and a high of 9.1


In [159]:
staten_island_rating = staten_island_best_of["rating"].mean()
staten_island_variety = len(staten_island_best_of.index)
staten_island_max = staten_island_best_of["rating"].max()
print('There are {} kinds of restaurants on Staten Island with an average rating of {} and a high of {}'.format(staten_island_variety, staten_island_rating, staten_island_max))

There are 10 kinds of restaurants on Staten Island with an average rating of 8.16 and a high of 8.7


In [160]:
bronx_rating = bronx_best_of["rating"].mean()
bronx_variety = len(bronx_best_of.index)
bronx_max = bronx_best_of["rating"].max()
print('There are {} kinds of restaurants the Bronx with an average rating of {} and a high of {}'.format(bronx_variety, bronx_rating, bronx_max))

There are 8 kinds of restaurants the Bronx with an average rating of 8.549999999999999 and a high of 9.2


# Conclusions

If you want the most variety in the kinds of the food you eat, go to Manhattan. If you want to go to the best of the best restaurants go to Brooklyn or the Bronx. If you want your average dining experience to be the best it can be go to the Bronx, but Brooklyn is not far behind.